# For Generation of Intrinio based Financial Data - Shankar

In [1]:
!pwd

/Users/NatarajanShankar/UC_Berkeley/Final_term/Capstone/W210/MIDS_capstone/metadata


In [2]:
from api_key import INTRINIO_USERNAME
from api_key import INTRINIO_PASSWORD             

import md_intrinio_client
import pandas as pd
import json

from md_intrinio_client import intrinio_get_company_metadata
from md_intrinio_client import intrinio_get_company_financials
from md_intrinio_client import intrinio_get_company_financials_csv
from md_intrinio_client import get_SandP_metadata
from md_intrinio_client import test_SandP_metadata
from finsymbols import symbols

import sys
import json
import finsymbols
import ast
import requests

In [3]:
import json 
SandP500 = {}
companyList = []
with open("SandP500_symbols.txt", "r") as fr:
            for line in fr:
                    company = json.loads(line)
                    SandP500[company["symbol"]] = line
                    companyList.append(company["symbol"])

tickerchunks = [companyList[x:x+100] for x in xrange(0, len(companyList), 100)]

In [4]:
print(len(tickerchunks[1]))
print((tickerchunks[1]))

100
[u'CTL', u'CERN', u'CF', u'SCHW', u'CHTR', u'CHK', u'CVX', u'CMG', u'CB', u'CHD', u'CI', u'XEC', u'CINF', u'CTAS', u'CSCO', u'C', u'CFG', u'CTXS', u'CLX', u'CME', u'CMS', u'COH', u'KO', u'CTSH', u'CL', u'CMCSA', u'CMA', u'CAG', u'CXO', u'COP', u'ED', u'STZ', u'COO', u'GLW', u'COST', u'COTY', u'CCI', u'CSRA', u'CSX', u'CMI', u'CVS', u'DHI', u'DHR', u'DRI', u'DVA', u'DE', u'DLPH', u'DAL', u'XRAY', u'DVN', u'DLR', u'DFS', u'DISCA', u'DISCK', u'DISH', u'DG', u'DLTR', u'D', u'DOV', u'DWDP', u'DPS', u'DTE', u'DRE', u'DUK', u'DXC', u'ETFC', u'EMN', u'ETN', u'EBAY', u'ECL', u'EIX', u'EW', u'EA', u'EMR', u'ETR', u'EVHC', u'EOG', u'EQT', u'EFX', u'EQIX', u'EQR', u'ESS', u'EL', u'ES', u'RE', u'EXC', u'EXPE', u'EXPD', u'ESRX', u'EXR', u'XOM', u'FFIV', u'FB', u'FAST', u'FRT', u'FDX', u'FIS', u'FITB', u'FE', u'FISV']


In [5]:
def new_intrinio_get_company_financials(symbol, year, quarter):
        # Get the latest FY Income Statement for "symbol"
        # 'type': 'FY'
        cleanedupdata = {}
        base_url = "https://api.intrinio.com"
        request_url = base_url + "/financials/standardized"
        query_params = {
                'ticker': symbol,
                'statement': 'income_statement',
                'fiscal_year' : str(year),
                'fiscal_period' : quarter
        }

        response = requests.get(request_url, params=query_params, auth=(INTRINIO_USERNAME, INTRINIO_PASSWORD))
        if response.status_code == 401: print("Unauthorized! Check your username and password."); exit()

        if response.status_code == 429:
            print("API query limit reached")
            return
        data = response.json()['data']


        for row in data:
                tag = row['tag']
                value = row['value']
                cleanedupdata["AASYMBOL"] = symbol
                cleanedupdata["ABYEAR"] = year
                cleanedupdata["ACPeriod"] = quarter
                cleanedupdata[tag] = value

        datalist=[]
        for key, value in sorted(cleanedupdata.items()):
            datalist.append(str(value))
        

        return(datalist)

In [6]:
def updated_cleanupdata(cleanedupdata):
    newData = {}
    newData['AASYMBOL'] = cleanedupdata['AASYMBOL']
    newData['ABYEAR'] = cleanedupdata['ABYEAR']
    newData['ACPeriod'] = cleanedupdata['ACPeriod']
    newData['basicdilutedeps'] = cleanedupdata.get('basicdilutedeps', 0.0)
    newData['basiceps'] = cleanedupdata.get('basiceps', 0.0)
    
    newData['cashdividendspershare'] = cleanedupdata.get('cashdividendspershare', 0.0)
    newData['dilutedeps'] = cleanedupdata.get('dilutedeps', 0.0)
    newData['incometaxexpense'] = cleanedupdata.get('incometaxexpense', 0.0)
    newData['netincome'] = cleanedupdata.get('netincome', 0.0)
    newData['netincomecontinuing'] = cleanedupdata.get('netincomecontinuing', 0.0)
    
    newData['netincomediscontinued'] = cleanedupdata.get('netincomediscontinued', 0.0)
    newData['netincometocommon'] = cleanedupdata.get('netincometocommon', 0.0)
    newData['netincometononcontrollinginterest'] = cleanedupdata.get('netincometononcontrollinginterest', 0.0)
    newData['operatingcostofrevenue'] = cleanedupdata.get('operatingcostofrevenue', 0.0)
    newData['operatingrevenue'] = cleanedupdata.get('operatingrevenue', 0.0)
    
    newData['othercostofrevenue'] = cleanedupdata.get('othercostofrevenue', 0.0)
    newData['otherincome'] = cleanedupdata.get('otherincome', 0.0)
    newData['preferreddividends'] = cleanedupdata.get('preferreddividends', 0.0)
    newData['sgaexpense'] = cleanedupdata.get('sgaexpense', 0.0)
    newData['totalcostofrevenue'] = cleanedupdata.get('totalcostofrevenue', 0.0)
    
    newData['totalgrossprofit'] = cleanedupdata.get('totalgrossprofit', 0.0)
    newData['totalinterestexpense'] = cleanedupdata.get('totalinterestexpense', 0.0)
    newData['totaloperatingexpenses'] = cleanedupdata.get('totaloperatingexpenses', 0.0)
    newData['totaloperatingincome'] = cleanedupdata.get('totaloperatingincome', 0.0)
    newData['totalotherincome'] = cleanedupdata.get('totalotherincome', 0.0)
    
    newData['totalpretaxincome'] = cleanedupdata.get('totalpretaxincome', 0.0)
    newData['totalrevenue'] = cleanedupdata.get('totalrevenue', 0.0)
    newData['weightedavebasicdilutedsharesos'] = cleanedupdata.get('weightedavebasicdilutedsharesos', 0.0)
    newData['weightedavebasicsharesos'] = cleanedupdata.get('weightedavebasicsharesos', 0.0)
    newData['weightedavedilutedsharesos'] = cleanedupdata.get('weightedavedilutedsharesos', 0.0)
    
    #newData[] = cleanedupdata.get(, 0.0)
    return newData

In [7]:
attributes = ["ticker", "year", "quarter", "basicdilutedeps", "basiceps",
              'cashdividendspershare', 'dilutedeps', 'incometaxexpense', 'netincome', 'netincomecontinuing',
              'netincomediscontinued', 'netincometocommon', 'netincometononcontrollinginterest', 'totalcostofrevenue', 
                  'operatingcostofrevenue', 'operatingrevenue',
              'othercostofrevenue', 'otherincome', 'preferreddividends', 'sgaexpense', 
              'totalgrossprofit', 'totalinterestexpense', 'totaloperatingexpenses', 'totaloperatingincome', 'totalotherincome', 
              'totalpretaxincome', 'totalrevenue', 'weightedavebasicdilutedsharesos', 'weightedavebasicsharesos', 'weightedavedilutedsharesos'
                 ]
print(len(attributes))

xx = ",".join(attributes)
print(xx)

30
ticker,year,quarter,basicdilutedeps,basiceps,cashdividendspershare,dilutedeps,incometaxexpense,netincome,netincomecontinuing,netincomediscontinued,netincometocommon,netincometononcontrollinginterest,totalcostofrevenue,operatingcostofrevenue,operatingrevenue,othercostofrevenue,otherincome,preferreddividends,sgaexpense,totalgrossprofit,totalinterestexpense,totaloperatingexpenses,totaloperatingincome,totalotherincome,totalpretaxincome,totalrevenue,weightedavebasicdilutedsharesos,weightedavebasicsharesos,weightedavedilutedsharesos


In [8]:
def updated_intrinio_get_company_financials(symbol, year, quarter):
        # Get the latest FY Income Statement for "symbol"
        # 'type': 'FY'
        cleanedupdata = {}
        base_url = "https://api.intrinio.com"
        request_url = base_url + "/financials/standardized"
        query_params = {
                'ticker': symbol,
                'statement': 'income_statement',
                'fiscal_year' : str(year),
                'fiscal_period' : quarter
        }

        response = requests.get(request_url, params=query_params, auth=(INTRINIO_USERNAME, INTRINIO_PASSWORD))
        if response.status_code == 401: print("Unauthorized! Check your username and password."); exit()

        if response.status_code == 429:
            print("API query limit reached")
            return
        data = response.json()['data']

        #print(data['basicdilutedeps'])
        for row in data:
                #print(row)
                tag = row['tag']
                value = row['value']

                cleanedupdata[tag] = value

        datalist=[]
        attr = []
        cleanedupdata["AASYMBOL"] = symbol
        cleanedupdata["ABYEAR"] = year
        cleanedupdata["ACPeriod"] = quarter
        cleanedupdata = updated_cleanupdata(cleanedupdata)
        for key, value in sorted(cleanedupdata.items()):
            datalist.append(str(value))
            attr.append(str(key))
 
        return(datalist, attr)
#data = intrinio_get_company_financials('GE', '2010', 'FY')

In [10]:
def generate_financial_data():
    SandP500 = {}
    companyList = []
    with open("SandP500_symbols.txt", "r") as fr:
            for line in fr:
                    company = json.loads(line)
                    SandP500[company["symbol"]] = line
                    companyList.append(company["symbol"])

    tickerchunks = [companyList[x:x+100] for x in xrange(0, len(companyList), 100)]


    years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
    quarters = ["Q1", "Q2", "Q3", "Q4", "FY"]
    #years = [2010]
    #quarters = ["Q1", "Q2"]
    #companies = ["GE", "CSCO", "GOOG", "FACE"]

    bigDict = {}

    attributes = ["ticker", "year", "quarter", "basicdilutedeps", "basiceps",
              'cashdividendspershare', 'dilutedeps', 'incometaxexpense', 'netincome', 'netincomecontinuing',
              'netincomediscontinued', 'netincometocommon', 'netincometononcontrollinginterest', 'totalcostofrevenue', 
                  'operatingcostofrevenue', 'operatingrevenue',
              'othercostofrevenue', 'otherincome', 'preferreddividends', 'sgaexpense', 
              'totalgrossprofit', 'totalinterestexpense', 'totaloperatingexpenses', 'totaloperatingincome', 'totalotherincome', 
              'totalpretaxincome', 'totalrevenue', 'weightedavebasicdilutedsharesos', 'weightedavebasicsharesos', 'weightedavedilutedsharesos'
                 ]
    print(len(attributes))

    xx = ",".join(attributes) + "\n"

    # tickerchunks[1], tickerchunks[2], tickerchunks[3] and tickerchunks[5] are done
    # 
    loopIndex = 0
    for company in tickerchunks[1]:
        with open("./revenue/"+company+"_Financials_by_Quarter.csv", 'w') as fw:
            fw.write(xx)
            print("working on {} - {}".format(company, loopIndex))
#             if loopIndex > 2:
#                 break
            loopIndex += 1
            #print("working on company {}".format(company))
            for year in years:

                #print("working on year {}".format(year))
                for quarter in quarters:
                    #print("working on quarter {}".format(quarter))

                    data, _ = updated_intrinio_get_company_financials(company, str(year), quarter)
                    yy = ",".join(data)
                    yy +="\n"

                    fw.write(yy)

            print("DONE with {}".format(company))      

In [11]:
generate_financial_data()

30
working on CTL - 0
DONE with CTL
working on CERN - 1
DONE with CERN
working on CF - 2
DONE with CF
working on SCHW - 3
DONE with SCHW
working on CHTR - 4
DONE with CHTR
working on CHK - 5
DONE with CHK
working on CVX - 6
DONE with CVX
working on CMG - 7
DONE with CMG
working on CB - 8
DONE with CB
working on CHD - 9
DONE with CHD
working on CI - 10
DONE with CI
working on XEC - 11
DONE with XEC
working on CINF - 12
DONE with CINF
working on CTAS - 13
DONE with CTAS
working on CSCO - 14
DONE with CSCO
working on C - 15
DONE with C
working on CFG - 16
DONE with CFG
working on CTXS - 17
DONE with CTXS
working on CLX - 18
DONE with CLX
working on CME - 19
DONE with CME
working on CMS - 20
DONE with CMS
working on COH - 21
DONE with COH
working on KO - 22
DONE with KO
working on CTSH - 23
DONE with CTSH
working on CL - 24
DONE with CL
working on CMCSA - 25
DONE with CMCSA
working on CMA - 26
DONE with CMA
working on CAG - 27
DONE with CAG
working on CXO - 28
DONE with CXO
working on COP 

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [56]:
import os
os.chdir("/Users/NatarajanShankar/UC_Berkeley/Final_term/Capstone/W210/MIDS_capstone/metadata")
!pwd

/Users/NatarajanShankar/UC_Berkeley/Final_term/Capstone/W210/MIDS_capstone/metadata


In [52]:
import glob, os
import ast

def convert_json_to_csv():
    companyDict = {}
    os.chdir("./data")
    #print("got here")
    for datafile in glob.glob("*Financials*.json"):
        #print(datafile)
        with open(datafile, "r") as fr:
            #print ("Processing {}".format(datafile))
            for line in fr:
                companyDict = ast.literal_eval(line)
                
                # Save company ticker
                for company, companyvalue in companyDict.items():
                    #print("Company is")
                    #print("{}".format(company, end=''))
                    # For every year subset
                    if isinstance(companyvalue, dict):
                        for year, yearvalue in companyvalue.items():
                            #print("Year is")
                            #print("{}".format(year, end=''))
                            # Look at the corresponding quarter
                            if isinstance(yearvalue, dict):
                                datalist=[]
                                # Look at company quarter
                                for quarter, quartervalue in yearvalue.items():
                                    #print("Quarter is")
                                    #print("{}".format(quarter, end=''))
                                    for financialdata in quartervalue.values():
                                        datalist.append(financialdata)
                                    
                                    if len(datalist) > 21:
                                        datalist[14] = "IGNORE"
                                        datalist[21] = "IGNORE"
                                    
                                    if len(datalist) == 2:
                                        datalist[0] = ""
                                        datalist[1] = ""
                                    print("{},{},{},{}".format(company, year, quarter, ",".join(datalist), end=''))
                                    
                                    datalist = []
        break
    
    

In [53]:
convert_json_to_csv()

FLIR,2010,Q1,
FLIR,2010,Q4,


In [ ]:
def get_company_financial_data_all(symbol):
    companyDict = {}
    with open("./data/Financials.json", "r") as fp:
        #companyDict = json.load(fp)
        for line in fp:
            companyDict = ast.literal_eval(line)

        for ticker in companyDict.keys():
            if ticker == symbol:
                print symbol, companyDict[symbol]
            

In [ ]:
get_company_financial_data_all("GE")

In [ ]:
def get_company_financial_data_by_year(symbol, year):
    companyDict = {}
    with open("./data/Financials.json", "r") as fp:
        #companyDict = json.load(fp)
        for line in fp:
            companyDict = ast.literal_eval(line)

        for ticker in companyDict.keys():
            if ticker == symbol:
                print symbol, companyDict[symbol][year]
            

In [ ]:
get_company_financial_data_by_year("GE", "2014")

In [ ]:
def get_company_financial_data_by_year_and_attr(symbol, year, attribute):
    companyDict = {}
    with open("./data/Financials.json", "r") as fp:
        #companyDict = json.load(fp)
        for line in fp:
            companyDict = ast.literal_eval(line)

        for ticker in companyDict.keys():
            if ticker == symbol:
                print symbol, companyDict[symbol][year][attribute]
            

In [ ]:
get_company_financial_data_by_year_and_attr("GE", "2014", "basiceps")

In [ ]:
def generate_company_financial_data_csv(symbol):
    companyDict = {}
    with open("./data/Financials.json", "r") as fp:
        for line in fp:
            companyDict = ast.literal_eval(line)
            
        for ticker in companyDict.keys():
            if ticker == symbol:
                print symbol, companyDict
            
            
            